In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 5.62M/5.62M [00:00<00:00, 35.3MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [7]:


models_list_3=[
        "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
"FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",

# "bert",

]


In [8]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [9]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [10]:
all_csv

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [11]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [12]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [13]:
train_data=all_csv


In [14]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [15]:

train_data=train_data.fillna(0)

train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [16]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [17]:
train_data.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [18]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...,...
2763,2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [19]:
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [20]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [21]:
train_data['lang'].value_counts()

,count
lang,
eng,2768


In [22]:
sum(train_data['lang'].isna())

0

In [23]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [24]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [25]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [26]:
nan_rows["lang"].value_counts()

,count
lang,


In [27]:
nan_rows.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [28]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id   fear    joy  sadness  surprise   lang  clean_message
lang                                                                   
eng  0     False  False  False    False     False  False          False
     1     False  False  False    False     False  False          False
     2     False  False  False    False     False  False          False
     3     False  False  False    False     False  False          False
     4     False  False  False    False     False  False          False
...          ...    ...    ...      ...       ...    ...            ...
     2763  False  False  False    False     False  False          False
     2764  False  False  False    False     False  False          False
     2765  False  False  False    False     False  False          False
     2766  False  False  False    False     False  False          False
     2767  False  False  False    False     False  False          False

[2768 rows x 7 columns]


<ipython-input-28-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [29]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [30]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [31]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [32]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [33]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1938      276  2214
fear          936     1278  2214
joy          1661      553  2214
sadness      1520      694  2214
surprise     1533      681  2214


In [34]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [35]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([4.0109, 0.8662, 2.0018, 1.5951, 1.6256], device='cuda:0')

In [36]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [37]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


In [38]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [39]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [40]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [41]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [42]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [43]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [44]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [45]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [46]:
USE_LORA=False

In [47]:
from transformers import AutoModel

In [48]:


torch.cuda.empty_cache()



print(x)
tokenizer = AutoTokenizer.from_pretrained(x)
embedding_model = AutoModel.from_pretrained(x, output_hidden_states=True)

FacebookAI/xlm-roberta-large


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [49]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [50]:
# a=

In [51]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [52]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    embedding_model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if embedding_model.config.pad_token_id is None:
        print("setting model pad token")
        embedding_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'<pad>'

In [53]:
device

'cuda'

In [55]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [56]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )


In [92]:

# Batch extraction function
def extract_embeddings(texts, tokenizer, model, batch_size=32, max_length=256):
    dataset = TextDataset(texts, tokenizer, max_length=max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Extract CLS token embeddings
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [93]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, tokenizer, embedding_model, batch_size=32, max_length=MAX_LENGTH)
x_valid_embeddings = extract_embeddings(x_valid, tokenizer, embedding_model, batch_size=32, max_length=MAX_LENGTH)

Extracting embeddings: 100%|██████████| 18/18 [00:21<00:00,  1.20s/it]


In [94]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)
y_valid_flat = np.array(y_valid)

In [96]:
import xgboost as xgb

In [97]:
xgb_models = {}
predictions = {}

for i, label in enumerate(label_columns):
    print(f"Training XGBoost for label: {label}")
    # Initialize and train XGBoost classifier
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        tree_method='gpu_hist' if device == "cuda" else 'hist'
    )
    xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
    xgb_models[label] = xgb_model

    # Predict on validation data
    predictions[label] = xgb_model.predict(x_valid_embeddings)

Training XGBoost for label: anger


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:48] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:50] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:50] WARNING: /workspace/src/common/

Training XGBoost for label: fear


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Training XGBoost for label: joy


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:54] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:56] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Training XGBoost for label: sadness


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:09:59] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Training XGBoost for label: surprise


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:10:00] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:10:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:10:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [98]:
# Evaluate the models
results = []
for i, label in enumerate(label_columns):
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_valid_flat[:, i],
        predictions[label],
        average="binary"
    )
    accuracy = accuracy_score(y_valid_flat[:, i], predictions[label])
    results.append({
        "Label": label,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Accuracy": accuracy
    })

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [99]:
# Display evaluation results
results_df = pd.DataFrame(results)
print(results_df)

      Label  Precision    Recall  F1 Score  Accuracy
0     anger   0.000000  0.000000  0.000000  0.897112
1      fear   0.665000  0.798799  0.725784  0.637184
2       joy   0.800000  0.066116  0.122137  0.792419
3   sadness   0.600000  0.163043  0.256410  0.685921
4  surprise   0.584615  0.240506  0.340807  0.734657


In [100]:
print(f"{x} evaluates:")

for k, res in enumerate(results):

  print(f"Class '{res['Label']}': Precision: {res['Precision']:.4f}, Recall: {res['Recall']:.4f}, F1: {res['F1 Score']:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

FacebookAI/xlm-roberta-large evaluates:
Class 'anger': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'fear': Precision: 0.6650, Recall: 0.7988, F1: 0.7258
Class 'joy': Precision: 0.8000, Recall: 0.0661, F1: 0.1221
Class 'sadness': Precision: 0.6000, Recall: 0.1630, F1: 0.2564
Class 'surprise': Precision: 0.5846, Recall: 0.2405, F1: 0.3408
Overall accuracy: 0.7347
-------------------------------------------------------------------------



In [101]:
# Save the XGBoost models if needed
for label, model in xgb_models.items():
    model.save_model(f"xgb_model_{label}.json")

In [104]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()

    if len(lang_valid_data) == 0:
        continue

    # Extract embeddings for the current language's validation data
    lang_x_valid_embeddings = extract_embeddings(lang_x_valid, tokenizer, embedding_model, max_length=MAX_LENGTH)
    print()
    # Initialize metrics storage
    all_precisions, all_recalls, all_f1_scores, all_accuracies = [], [], [], []

    # Evaluate each label separately
    for i, label in enumerate(label_columns):
        # Predict using the corresponding XGBoost model
        lang_predictions = xgb_models[label].predict(lang_x_valid_embeddings)

        # Compute metrics
        precision, recall, f1, _ = precision_recall_fscore_support(
            np.array(lang_y_valid)[:, i],  # True labels for this label
            lang_predictions,
            average="binary",
            zero_division=0
        )
        accuracy = accuracy_score(np.array(lang_y_valid)[:, i], lang_predictions)

        # Store metrics
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1_scores.append(f1)
        all_accuracies.append(accuracy)

    # Aggregate metrics (average across all labels for the language)
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [np.mean(all_precisions)],
        "Recall": [np.mean(all_recalls)],
        "F1_score": [np.mean(all_f1_scores)],
        "Accuracy": [np.mean(all_accuracies)]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)


Extracting embeddings: 100%|██████████| 18/18 [00:23<00:00,  1.28s/it]


  Language  Precision    Recall  F1_score  Accuracy
0      eng   0.529923  0.253693  0.289028  0.749458



<ipython-input-104-2f79ff4420ee>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


In [103]:
a=

SyntaxError: invalid syntax (<ipython-input-103-2d920c342ece>, line 1)